# Genetic Algorithm Library

An individual is encoded as a string of `l` binary digits. 

For example, suppose `l=8`.
An individual in the population could be `10100111`
and the target would be `11111111`

In [4]:
import numpy as np
import matplotlib.pyplot as plt

## Define Objective Fitness
We are trying to minimize the Hamming distance between some binary string `x` and an all-ones binary string `y`. This is equivalent to maximizing the number of `1`s in `x`.

In [5]:
def hamming_dist(x, y):
    ''' Computes the Hamming distance
    '''
    
    n_unequal = sum(x - y == 0)
    n_tot = len(x)
    
    return n_unequal / n_tot

def fitness(x):
    ''' The fitness objective
    '''
    
    target = np.ones(x.shape)
    
    return hamming_dist(x, target)

## Initialization

In [328]:
def create_population(size, genotype_size=8):
    '''Randomly initializes population
    
    genotype_size: number of bits
    '''
    
    return np.random.randint(2, size=(size, genotype_size), dtype=np.uint8)

population = create_population(4)
fitnesses = np.array([fitness(parent) for parent in population])
population

array([[0, 1, 0, 0, 1, 0, 1, 0],
       [1, 1, 0, 0, 1, 0, 1, 1],
       [0, 0, 0, 1, 0, 1, 0, 1],
       [0, 1, 0, 0, 0, 0, 0, 0]], dtype=uint8)

## Selection

In [7]:
def argselect_uniform(population, k):
    '''Uniform Stochastic Selection
    
    Select the arguments of k individuals 
    with replacement from the population uniformly 
    at random.
    '''
    pop_size = population.shape[0] 
    
    ind = np.random.choice(pop_size, size=k, replace=True)

    return ind


def select_uniform(population, k):
    '''Uniform Stochastic Selection
    
    Select k individuals with replacement from the 
    population uniformly at random.
    '''

    ind = argselect_uniform(population, k)
    individuals = population[ind]
    
    return individuals

def select_fitness_proportional(population, fitnesses, k):
    '''Fitness-Proportional Selection
    
    Select k individuals with replacement from the population 
    uniformly at random proportional to the fitness of 
    the individual. This is also known as roulette-wheel 
    selection or stochastic sampling with replacement.
    '''
    pop_size = population.shape[0] 
    
    probabilities = fitnesses / np.sum(fitnesses)
    ind = np.random.choice(pop_size, size=k, replace=True, p=probabilities)
    individuals = population[ind]
    
    return individuals

def select_stochast_univ_samp():
    ''' Stochastic Universal Sampling
    '''
    # TODO: implement
    return None

def select_sigma_scaling(population, fitnesses, k):
    '''Sigma scaling selection
    '''
    pop_size = population.shape[0] 
    
    sigma = np.std(fitnesses)
    expected_cnts = np.empty(pop_size)
    if sigma > 0.0001:
        expected_cnts[:] = 1 + (fitnesses - np.mean(fitnesses)) / sigma
    else:
        expected_cnts[:] = 1
    
    max_exp_cnt = 1.5
    min_exp_cnt = 0
    
    expected_cnts[expected_cnts > max_exp_cnt] = max_exp_cnt
    expected_cnts[expected_cnts < min_exp_cnt] = min_exp_cnt
    
    probabilities = expected_cnts / np.sum(expected_cnts)
    ind = np.random.choice(pop_size, size=k, replace=True, p=probabilities)
    individuals = population[ind]
    
    return individuals

def select_boltzmann(population, fitnesses, k, T, prev_pop_avg):
    '''Boltzmann Selection

    '''
    # TODO: implement
    return None

def select_k_best(population, fitnesses, k):
    '''Truncation selection 
    
    Select k best from population according to fitnesses.
    
    k: number of individuals
    '''
    
    top_k_idxs = np.argpartition(fitnesses, -k)[-k:]
    individuals = population[top_k_idxs]
    
    return individuals

def select_linear_ranking(population, fitnesses, k):
    '''Linear Ranking Selection
    
    Select k individuals with replacement from the population 
    uniformly at random proportional to the relative fitness 
    ranking of the individual.
    '''
    pop_size = population.shape[0] 
    
    rankings_asc = np.argsort(np.argsort(fitnesses)) + 1
    probabilities = rankings_asc / np.sum(rankings_asc)

    ind = np.random.choice(pop_size, size=k, replace=True, p=probabilities)
    
    individuals = population[ind]
    
    return individuals

def select_exponetial_ranking(population, fitnesses, k, c=0.99):
    '''Exponential Ranking Selection
    '''
    if c <= 0 or c >= 1:
        raise ValueError("0 < c < 1 must hold")
        
    pop_size = population.shape[0] 

    rankings_asc = np.argsort(np.argsort(fitnesses))
    probabilities = (c ** (pop_size - rankings_asc)) / np.sum(c ** (pop_size - rankings_asc))

    ind = np.random.choice(pop_size, size=k, replace=True, p=probabilities)

    individuals = population[ind]

    return individuals

def select_tournament(population, fitnesses, k, n_way=2):
    '''Tournament Selection
    Uniformly at random select `n_way` individuals from the 
    population then selecting the best (or worst) individual
    from the `n_way` competitors as the winner (or loser). There
    will be `k` tournaments run with replacement on the 
    population.
    
    Notes
    -----
    Binary tournaments (`n_way` = 2) are equivalent to linear 
    ranking selection in expectation. If `n_way` = 3, it is 
    equivalent, in expectation, to quadratic ranking selection.
    '''
    
    if n_way < 2 or n_way > len(population):
        raise ValueError("The number of competitors in the \
                         tournament must be greater than 1 and less \
                         than the number of individuals in the \
                         population")

    individuals = []
    for i in range(k):
        ind = argselect_uniform(population, n_way)
        winner = np.argmax(fitnesses[ind], axis=0)
        individuals.append(population[winner])
    
    return individuals

## Variation: Crossover & Mutation

In [ ]:
def vary(parents, n_offspring):
    ''' Perform both crossover and mutation
    '''
    
    offspring = crossover(parents, n_offspring)
    offpring = mutation(offspring)
    
    return offspring

### Crossover

In [5]:
def crossover(parents, n_offspring, cprob=1.):
    '''One-point crossover applied to all parents
    
    cprob: probability of crossover
    '''

    offspring = parents[:n_offspring].copy()

    for i in range(1, n_offspring, 2):
        if np.random.rand() < cprob:
            child_1, child_2 = crossover_one_point(parents[i - 1], parents[i])
            offspring[i - 1] = child_1
            offspring[i] = child_2
        
    return offspring


def crossover_one_point(parent_1, parent_2):
    '''One-point crossover
    '''
    
    L = parent_1.size
    crossover_point = np.random.randint(0, L)
    child_1 = np.hstack((parent_1[:crossover_point], parent_2[crossover_point:]))
    child_2 = np.hstack((parent_2[:crossover_point], parent_1[crossover_point:]))

    return child_1, child_2
    
def crossover_two_point(parent_1, parent_2):
    ''' Two-point crossover
    '''
    # TODO: Implement
    return crossover_n_point(parent_1, parent_2, n=2)

def crossover_n_point(parent_1, parent_2, n):
    ''' n-point crossover
    '''
    # TODO: use np.where instead
    
    L = parent_1.size
    crossover_points = sorted(np.random.choice(np.arange(0, L), size=n, replace=False))
    child_1 = parent_1.copy()
    child_2 = parent_2.copy()

    for i, cpoint in enumerate(crossover_points):
        if i == len(crossover_points) - 1:
            break
        next_cpoint = crossover_points[i + 1]
        if i % 2 == 0:
            child_1[cpoint:next_cpoint] = parent_2[cpoint:next_cpoint]
        else:
            child_2[cpoint:next_cpoint ] = parent_1[cpoint:next_cpoint] 

    return child_1, child_2

def crossover_uniform(parent_1, parent_2):
    '''Uniform crossover
    '''
    # TODO: Implement
    raise NotImplementedError("crossover_uniform not yet implemented")

### Mutation

In [1]:
def mutation(offspring, mprob=1.):
    '''Bit-flip mutation applied to all offspring
    
    mprob: probability of mutation
    '''
    
    offspring_mut = []
    
    for child in population:
        if np.random.rand() < mprob:
            child = mutate(child)
            
        offspring_mut.append(child)
            
    return offspring_mut


def mutate_bit_flip(individual, gene_mut_prob=None):
    '''Bit-flip mutation
    '''
    indiv_mut = individual.copy()
    L = individual.size
    
    if gene_mut_prob is None:
        gene_mut_prob = 1/L
        
    mutated = np.random.uniform(0, 1, size=L) < gene_mut_prob
    indiv_mut = np.where(mutated, ~indiv_mut.astype(bool), indiv_mut)
    
    return indiv_mut

def mutate_interchange(individual):
    '''Interchange mutation
    '''
    indiv_mut = individual.copy()
    L = indiv_mut.size
    ind = np.random.randint(0, L, size=2)
    # swap first and second genes
    indiv_mut = swap(indiv_mut, ind[0], ind[1])

    return indiv_mut

def mutate_reverse(individual):
    '''Reverse mutation
    '''
    indiv_mut = individual.copy()
    L = indiv_mut.size
    ind = np.random.randint(0, L)
    indiv_mut = swap(indiv_mut, ind, ind - 1)
    
    return indiv_mut

def mutate_gaussian(individual):
    '''Gaussian mutation
    '''
    # TODO: Implement
    raise NotImplementedError("mutate_gaussian not yet implemented")

def mutate_boundary(individual):
    '''Boundary mutation
    '''
    # TODO: Implement
    raise NotImplementedError("mutate_boundary not yet implemented")

def mutate_uniform(individual):
    '''Uniform mutation
    '''
    # TODO: Implement
    raise NotImplementedError("mutate_uniform not yet implemented")

def mutate_non_uniform(individual):
    '''Non-Uniform mutation
    '''
    # TODO: Implement
    raise NotImplementedError("mutate_non_uniform not yet implemented")

def mutate_shrink(individual):
    '''Shrink mutation
    '''
    # TODO: Implement
    raise NotImplementedError("mutate_shrink not yet implemented")

#### Helper Functions

In [448]:
def swap(arr, idx_1, idx_2):
    '''Swap two array elements
    '''
    arr_copy = arr.copy()
    first = arr_copy[idx_1]
    second = arr_copy[idx_2]
    arr_copy[idx_1] = second
    arr_copy[idx_2] = first
    
    return arr_copy

## Run Genetic Algorithm

In [1]:
def plot_results(best_so_far):
    # First, display the maximum fitness value at each generation
    plt.title('Best-So-Far Curve')
    plt.plot(best_so_far)
    plt.xlabel('Generation')
    plt.ylabel('Fitness Best So Far')
    plt.show()

def run_genetic_algo(n_individuals, fitness, genotype_size=64, budget=100, display_results=True):
    population = create_population(n_individuals, genotype_size=64)

    # store an extra value because we want to store the max before and after running the GA
    best_so_far = np.zeros(budget + 1)

    for i in range(budget):
        fitnesses = [fitness(parent) for parent in population]
        # save the maximum fitness for each generation
        best_so_far[i] = max(fitnesses)

        parents = select_parents(population, fitnesses, n_parents=n_individuals)

        offspring = vary(parents, n_offspring=10)

        population = select_survivors(offspring, fitnesses, k=10)

    # recompute fitnesses for the last generation
    fitnesses = [fitness(parent) for parent in population]
    best_so_far[-1] = max(fitnesses)
    if display_results:
        plot_results(best_so_far)

n_individuals = 10
run_genetic_algo(n_individuals, fitness, genotype_size=64, budget=1000, 
                 display_results=True)

NameError: name 'fitness' is not defined